Первое задание

In [ ]:
from math import erf, sqrt, pi, exp

# нормальное распределение функции
def f_norm(x, mu=0, s=1):
    return (1+erf((x-mu)/sqrt(2)/s))/2

# нормальное распределение плотность
def rho_norm(x, mu=0,s=1):
    return 1/sqrt(2*pi*s)*exp(-(x-mu)**2/2/s**2)

# вероятность ошибки первого рода
def p_value(x, mu=0, s=1):
    return 2*(1-f_norm(x,mu,s)) if x >= mu else 2*f_norm(x,mu,s)

# Обратная функция распределения
def inv_f_norm(p, mu, s, t=0.001):
    if mu != 0 or s != 1:
        return mu + s *inv_f_norm(p,0,1,t)
    low_x, low_p = -100, 0
    hi_x, hi_p = 100, 1
    while hi_x - low_x > t:
        mid_x = (low_x + hi_x)/2
        mid_p = f_norm(mid_x)
        if mid_p < p:
            low_x, low_p = mid_x, mid_p
        elif mid_p>p:
            hi_x,hi_p = mid_x,mid_p
        else:
            break
    return mid_x

def test(x,n, p0=4/8, pa=3/8):
    alpha = 0.05
    beta = 0.92
    # вычисляю mu
    mu0 = n*p0
    mu1 = n*pa
    # нахожу стандартное отклонение
    sigma0=sqrt(n*p0*(1-p0))
    sigmaa=sqrt(n*pa*(1-pa))
    # нахожу нижнюю и верхнюю критическую точку 
    slow = inv_f_norm(alpha/2,mu0,sigma0)
    shigh=2*mu0-slow
    # формула из методы
    mosh_prov = 1-f_norm(shigh,mu1,sigmaa)+f_norm(slow,mu1,sigmaa)
    p = p_value(x/8,mu0,sigma0)
    if p>=alpha and 1-f_norm(shigh,mu1,sigmaa)+f_norm(slow,mu1,sigmaa)>=beta:
        print("Нулевая гипотеза не опровергнута")
        print(f"Мощность проверки равна {mosh_prov}")
        return p
    return False

def first_task(j):
    # находим идеально кол-во чассов для заданного количетсва плитки
    for i in range(100,10000):
        h3 = test(j,i,3/8,4/8)
        h4 = test(j,i)
        # если обе гипотезы верны сравниваем какая из них ближе к мю(сравниваем p_value)
        if bool(h3) and bool(h4) and h3>h4:print(f"(обе теории верны)Плиточник выкладывает 3 кв метра плитки в час\nКоличество часов {i}");break
        elif bool(h3) and bool(h4):print(f"(обе теории верны)Плиточник выкладывает 4 кв метра плитки в час\nКоличество часов {i}");break
        # если верна только одна то выводим только её
        if bool(h4):print(f"Плиточник выкладывает 4 кв метра плитки в час\nКоличество часов {i}");break
        if bool(h3):print(f"Плиточник выкладывает 3 кв метра плитки в час\nКоличество часов {i}");break
            

def main():
    first_task(700)

if __name__ == "__main__":
    main()

Второе задание

In [ ]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import solve as gauss
from statistics import mean
from math import sqrt
from first_task import f_norm, inv_f_norm, p_value
from statistics import mean

def GetData(data=[]):
    url = 'https://cryptocharts.ru/bitcoin-dollar/'
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')

    table_class = 'TableRates'
    table = soup.find('table', class_=table_class)

    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        for cell in cells[1::2]:
            data.insert(0, cell.text.strip())

# дисперсия
def variance(x):
    return mean(list(map(lambda t: t ** 2, x)))

# стандартное отклонение
def standard_deviation(x):
    return sqrt(variance(x))

def find_max_by_abs(nums: list[float]) -> float:
    max_num = 0
    for num in nums:
        if abs(num) > abs(max_num):
            max_num = num
    return abs(max_num)

# Полиномиальная аппроксимация
def approx_poly(x, t, r):
    M = [[] for _ in range(r + 1)]
    b = []
    for l in range(r + 1):
        for q in range(r + 1):
            M[l].append(sum(list(map(lambda z: z ** (l + q), t))))
        b.append(sum(xi * ti ** l for xi, ti in zip(x, t)))
    a = gauss(np.array(M, dtype="float64"), np.array(b, dtype="float64"))
    return a

def test(x, n):
    alpha =0.05
    dataset = x.copy()
    t = list(range(len(dataset)))
    # Аппроксимация
    A_0 = approx_poly(dataset, t, n)
    A_1 = approx_poly(dataset,t,n)
    # Вычисляем аппроксимированные значения
    x_0_approx = [sum(A_0[i] * ti ** i for i in range(len(A_0))) for ti in t]
    x_1_approx = [sum(A_1[i] * ti ** i for i in range(len(A_0))) for ti in t]
    # Вычисление остатков (ошибок аппроксимации)
    epsilon0 = [x_0_approx[i]-dataset[i] for i in range(len(dataset))]
    epsilon1 = [x_1_approx[i]-dataset[i] for i in range(len(dataset))]
    # Средние значения и стандартные отклонения
    mu0 = mean(epsilon0)
    mu1 = mean(epsilon1)
    # стандартное отклонение
    sigma0 = standard_deviation(epsilon0)
    sigma1 = standard_deviation(epsilon1)
    # Наибольшее отклонение
    max_dev0 = find_max_by_abs([i - mu0 for i in epsilon0])
    # Вычисление p-значения
    slow = inv_f_norm(alpha/2,mu0,sigma0)
    shigh=2*mu0-slow
    return p_value(max_dev0+mu0, 0,sigma0), x_0_approx, 1-f_norm(shigh,mu1,sigma1)+f_norm(slow,mu1,sigma1)
# f_norm118 - f_norm82

def main():
    # Получение данных
    dataset = []
    GetData(dataset)
    dataset = [float(value.replace(',', '.')) for value in dataset]  # Преобразуем строки в числа
    max_p = -1
    max_n = 0
    max_w=0
    x_appr = []
    for i in range(5,50):
        p, x_appr, w = test(dataset,i)
        if max_p<p:
            max_p=p
            max_w = w
            max_n=i
    t = list(range(len(dataset)))
    print(f"Лучшее p_value:{max_p}")
    print(f"Его w:{max_w}")
    # Визуализация данных и аппроксимаций
    plt.figure(figsize=(10, 6))
    plt.plot(t, dataset, label='Исходные данные', marker='o')
    plt.plot(t, x_appr, label=f'Аппроксимированная кривая {max_n} степени', linestyle='-')
    plt.xlabel('Время')
    plt.ylabel('Значение')
    plt.title('Аппроксимация полиномом')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()
